___
# Ciência dos Dados - PROJETO 2

___
## Rafael dos Santos

## José Antônio Bechara

## Arthur Alegro de Oliveira

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [9]:
import pandas as pd
import os
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
stemmer = SnowballStemmer("english")

In [16]:
def cleaner_not_alphanumeric(dataframe, column, export=False):
    
    # Alphanumerics symbols:
    alpha_numerics = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
                      'v', 'w', 'x', 'y', 'z', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', ' ']
    
    comments_list = []
    
    # Reading all comments in a dataframe's column:
    for comment in dataframe[column]:
        comment = comment.lower()
        # Selecting only alphanumeric characters:
        char_list = [char for char in comment if char in alpha_numerics]
        # Generating clean comment:
        clean_comment = ''.join(char_list)
        
        clean_comment_stemizados = [stemmer.stem(palavra) for palavra in clean_comment]
        comments_list.append(clean_comment_stemizados)
                          
    dataframe[column] = comments_list
    
    if export == False:
        return dataframe
    elif export == True:
        dataframe.to_excel("nome.xlsx")

In [17]:
data = pd.read_excel("spamham2019.xlsx")

In [18]:
df = cleaner_not_alphanumeric(data, "Email", export=True)

In [20]:
training_data, testing_data = train_test_split(pd.read_excel('nome.xlsx'), test_size=0.25)

In [21]:
training_percentile_not_spam = len(training_data.loc[training_data["Class"]=='ham'])/ len(data)
training_percentile_spam = 1 - training_percentile_not_spam
total_training = len(data)

___
## 3. Classificador Naive-Bayes

In [22]:
list_words_spam = []
list_words_not_spam = []

spam_frequency = {}
not_spam_frequency = {}

i = 0
for email in testing_data:
    for word in email:
        if list(testing_data["Class"])[i] == "spam":
            if word not in list_words_spam:
                list_words_spam.append(word)
                spam_frequency[word] = 1
            else:
                spam_frequency[word] += 1
        else:
            if word not in list_words_not_spam:
                list_words_not_spam.append(word)
                not_spam_frequency[word] = 1
            else:
                not_spam_frequency[word] += 1
        i += 1
        
total_words = len(list_words_not_spam) + len(list_words_spam)

In [23]:
def calc_prob(email, dic_words, n_total_words_in_email, prob_a):
    P_b_dado_a = prob_a
    
    for word in email:
        ocurrence = 1
        if word in dic_palavras:
            ocorrencia += dic_words[word]
        p_a_dado_b = ocorrencia / (n_total_words_in_email+total_words)
    
    return p_a_dado_b

In [24]:
def comparador_prob(prob_0, prob_1):
    if prob_0 > prob_1:
        return 0
    else:
        return 1

___
## 4. Qualidade do Classificador alterando a base de treinamento